#### GPT

Reference:
- https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/chatgpt?tabs=python-new&pivots=programming-language-chat-completions

In [26]:
import yaml

file_path = "credentials.yaml"

with open(file_path, "r") as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [7]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = credentials["openai"]["key"],  
  api_version = credentials["openai"]["version"],
  azure_endpoint = credentials["openai"]["endpoint"]
)

response = client.chat.completions.create(
    model="35-model", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": "chatcmpl-8wR0YCq6zbGCFNjAdSok4f7Pb6aM0",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The founders of Microsoft are Bill Gates and Paul Allen. They established the company on April 4, 1975.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1708937534,
  "model": "gpt-35-turbo-16k",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 29,
    "total_tokens"

In [8]:
content = "Microsoft Copilot とは何ですか？"

In [10]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
endpoint = credentials["text_analytics"]["endpoint"]
key = credentials["text_analytics"]["key"]

text_analytics_client = TextAnalyticsClient(endpoint, AzureKeyCredential(key))

In [11]:
entities_result = text_analytics_client.recognize_entities([content], language="ja")

In [12]:
key_phrases_result = text_analytics_client.extract_key_phrases([content], language="ja")

In [13]:
entities_result

[RecognizeEntitiesResult(id=0, entities=[CategorizedEntity(text=Microsoft, category=Organization, subcategory=None, length=9, offset=0, confidence_score=0.81)], warnings=[], statistics=None, is_error=False, kind=EntityRecognition)]

In [14]:
key_phrases_result

[ExtractKeyPhrasesResult(id=0, key_phrases=['Microsoft Copilot'], warnings=[], statistics=None, is_error=False, kind=KeyPhraseExtraction)]

In [20]:
search_words = []

In [21]:
for idx, review in enumerate(entities_result):
    for entity in review.entities:
        search_words.append(entity.text)

In [22]:
search_words

['Microsoft']

In [24]:
for idx, doc in enumerate(key_phrases_result):
    if not doc.is_error:
        for key_phrase in doc.key_phrases:
            search_words.append(key_phrase)

In [25]:
search_words

['Microsoft', 'Microsoft Copilot']

In [27]:
subscription_key = credentials["web_search"]["key"]
assert subscription_key

In [28]:
search_url = "https://api.bing.microsoft.com/v7.0/search"

In [29]:
search_term = " ".join(search_words)

In [34]:
search_term

'Microsoft Microsoft Copilot'

In [30]:
import requests

In [31]:
headers = {"Ocp-Apim-Subscription-Key": subscription_key}
params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()

In [32]:
def get_reference(response: requests.models.Response) -> str:
    search_results = response.json()
    reference = ""
    for v in search_results["webPages"]["value"]:
        print(v["snippet"])
        reference += v["snippet"]
        reference += "\n"
    return reference

In [33]:
reference = get_reference(response)

<b>Copilot</b> を一部の <b>Microsoft</b> 365 アプリの中で使用する 1 これまで以上に優れた作成を実現 最適な <b>Copilot</b> エクスペリエンスを見つける <b>Copilot</b> 無料バージョンの <b>Copilot</b> をお試しください <b>Copilot</b> Pro 1 <b>Copilot</b> Pro を探索する <b>Copilot</b> for <b>Microsoft</b> 365 <b>Copilot</b> for <b>Microsoft</b> 365 を探索する エクスペリエンスを高める AI 搭載の検索 詳細情報 <b>Copilot</b> を使用してビジネスの未来を形成する さらに詳しく知る 最重要事項を確認する AI を使用してすべてのロールを強化する <b>Microsoft</b> <b>Copilot</b> for Sales を探索する
<b>Microsoft</b> <b>Copilot</b> leverages the power of AI to boost productivity, unlock creativity, and helps you understand information better with a simple chat experience. Get the app Sign in to experience <b>Microsoft</b> <b>Copilot</b>
*A <b>Microsoft</b> 365 Business Standard or <b>Microsoft</b> 365 Business Premium license is required to purchase <b>Copilot</b> for <b>Microsoft</b> 365. Enhance your experiences Whatever you’re into—browsing the web, searching for answers, exploring your creative potential, or coming up with more useful content, <b>Copilot

In [35]:
modified_content = "必要に応じてcontextの情報を参考にし、以下の質問に答えてください。\n" + content + "\n### context ###\n" + reference

In [36]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = credentials["openai"]["key"],  
  api_version = credentials["openai"]["version"],
  azure_endpoint = credentials["openai"]["endpoint"]
)

response = client.chat.completions.create(
    model="35-model", # model = "deployment_name".
    messages=[
        {"role": "user", "content": modified_content}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": "chatcmpl-8wRkb2TlzjLTR0EeFWeqk3QF0OYZU",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Microsoft Copilotは、AIの力を活用して生産性を向上させ、創造力を引き出し、シンプルなチャット体験で情報を理解するのに役立つものです。Microsoft 365 Business StandardまたはMicrosoft 365 Business Premiumライセンスが必要です。Copilotは、ウェブ閲覧、回答の検索、創造的なポテンシャルの探求、より有用なコンテンツの作成など、さまざまな体験を向上させるために役立ちます。",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1708940389,
  "model": "gpt-35-turbo-16k",
  "object": "chat.completi